reference: [Fine-Tune Wav2Vec2 for English ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-wav2vec2-english#set-up-trainer)

## 1.&nbsp;라이브러리 설치

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 22 08:36:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
%%capture
!apt install git-lfs

## 2.&nbsp;데이터 로드

In [ ]:
from datasets import load_dataset, load_metric, Audio
zeroth_data = load_dataset("kresnik/zeroth_korean", 'clean')

Downloading:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset zeroth_korean_asr downloaded and prepared to /root/.cache/huggingface/datasets/kresnik___zeroth_korean_asr/clean/1.0.1/f6cf96a53d5512525e3113bab8048d36ce268658d6e0c40d45f65dfa3f0bc343. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
zeroth_data

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 457
    })
})

## 3.&nbsp;데이터 전처리

In [ ]:
zeroth_data = zeroth_data.remove_columns(['speaker_id', 'chapter_id', 'id'])

In [ ]:
zeroth_data

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 457
    })
})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(zeroth_data["train"].remove_columns(["file", "audio"]), num_examples=10)

,text
0,북한이 괌을 사정권으로 하는 무수단 미사일을 쏘아 올린 지난달 이십 이 일 직전입니다
1,하지만 낯선 환경 때문인지 아니면 죽음을 예감한 건지 역주행하는 놈에 높게 쌓아진 두둑을 뛰어 오르는 놈에 꼼짝도 하지 않고 서서 꽥꽥 소리 지르는 놈에 야단법석이다
2,단지 특별법상에 아주 구체적인 내용으로 어떻게 구성을 하고 이런 내용은 없지만 분명히 준비할 수 있는 다리가 존재할 수 있도록 규정이 되어 있습니다
3,유종헌안녕들 하십니까 시작된 게시판 외부인이 대자보 훼손하다니대자보를 작성자에게 다시 돌려주고 나니 해당 대자보가 붙어있던 정경대 후문은 한바탕 난리가 나 있었다
4,올해 터를 매입할 계획이었으나 군의회에서 예산 승인이 나지 않았다
5,힐러리가 기침을 멈추지 않거나 계단을 오르다가 발을 삐끗해서 경호원의 부축을 받거나 어지럼증으로 휘청거리기라도 하면 그 때마다 미디어는 법석을 떨었다
6,광주지역의 장병완 박혜자 의원 등 당내 현역들은 야권 재편 등 상황을 예의주시하면서 거취를 고민하고 있다
7,하지만 다행히도 유방암은 치료 가능성이 매우 높은 암에 속합니다
8,하지만 이같은 노동절 휴일수당은 여전히 많은 노동자에게 그림의 떡인 상황이다
9,이러한 결과에 대해 거센 비난 여론이 쏟아지자 그리스 정부는 조세 포탈을 해온 예순 여덟 명의 의사를 고발했다


## 4.&nbsp;토크나이저

### Wav2Vec2CTCTokenizer

In [ ]:
__all__ = ["split_syllable_char", "split_syllables",
           "join_jamos", "join_jamos_char",
           "CHAR_INITIALS", "CHAR_MEDIALS", "CHAR_FINALS"]

import itertools

INITIAL = 0x001
MEDIAL = 0x010
FINAL = 0x100
CHAR_LISTS = {
    INITIAL: list(map(chr, [
        0x3131, 0x3132, 0x3134, 0x3137, 0x3138, 0x3139,
        0x3141, 0x3142, 0x3143, 0x3145, 0x3146, 0x3147,
        0x3148, 0x3149, 0x314a, 0x314b, 0x314c, 0x314d,
        0x314e
    ])),
    MEDIAL: list(map(chr, [
        0x314f, 0x3150, 0x3151, 0x3152, 0x3153, 0x3154,
        0x3155, 0x3156, 0x3157, 0x3158, 0x3159, 0x315a,
        0x315b, 0x315c, 0x315d, 0x315e, 0x315f, 0x3160,
        0x3161, 0x3162, 0x3163
    ])),
    FINAL: list(map(chr, [
        0x3131, 0x3132, 0x3133, 0x3134, 0x3135, 0x3136,
        0x3137, 0x3139, 0x313a, 0x313b, 0x313c, 0x313d,
        0x313e, 0x313f, 0x3140, 0x3141, 0x3142, 0x3144,
        0x3145, 0x3146, 0x3147, 0x3148, 0x314a, 0x314b,
        0x314c, 0x314d, 0x314e
    ]))
}
CHAR_INITIALS = CHAR_LISTS[INITIAL]
CHAR_MEDIALS = CHAR_LISTS[MEDIAL]
CHAR_FINALS = CHAR_LISTS[FINAL]
CHAR_SETS = {k: set(v) for k, v in CHAR_LISTS.items()}
CHARSET = set(itertools.chain(*CHAR_SETS.values()))
CHAR_INDICES = {k: {c: i for i, c in enumerate(v)}
                for k, v in CHAR_LISTS.items()}


def is_hangul_syllable(c):
    return 0xac00 <= ord(c) <= 0xd7a3  # Hangul Syllables


def is_hangul_jamo(c):
    return 0x1100 <= ord(c) <= 0x11ff  # Hangul Jamo


def is_hangul_compat_jamo(c):
    return 0x3130 <= ord(c) <= 0x318f  # Hangul Compatibility Jamo


def is_hangul_jamo_exta(c):
    return 0xa960 <= ord(c) <= 0xa97f  # Hangul Jamo Extended-A


def is_hangul_jamo_extb(c):
    return 0xd7b0 <= ord(c) <= 0xd7ff  # Hangul Jamo Extended-B


def is_hangul(c):
    return (is_hangul_syllable(c) or
            is_hangul_jamo(c) or
            is_hangul_compat_jamo(c) or
            is_hangul_jamo_exta(c) or
            is_hangul_jamo_extb(c))


def is_supported_hangul(c):
    return is_hangul_syllable(c) or is_hangul_compat_jamo(c)


def check_hangul(c, jamo_only=False):
    if not ((jamo_only or is_hangul_compat_jamo(c)) or is_supported_hangul(c)):
        raise ValueError(f"'{c}' is not a supported hangul character. "
                         f"'Hangul Syllables' (0xac00 ~ 0xd7a3) and "
                         f"'Hangul Compatibility Jamos' (0x3130 ~ 0x318f) are "
                         f"supported at the moment.")


def get_jamo_type(c):
    check_hangul(c)
    assert is_hangul_compat_jamo(c), f"not a jamo: {ord(c):x}"
    return sum(t for t, s in CHAR_SETS.items() if c in s)


def split_syllable_char(c):
    """
    Splits a given korean syllable into its components. Each component is
    represented by Unicode in 'Hangul Compatibility Jamo' range.

    Arguments:
        c: A Korean character.

    Returns:
        A triple (initial, medial, final) of Hangul Compatibility Jamos.
        If no jamo corresponds to a position, `None` is returned there.

    Example:
        >>> split_syllable_char("안")
        ("ㅇ", "ㅏ", "ㄴ")
        >>> split_syllable_char("고")
        ("ㄱ", "ㅗ", None)
        >>> split_syllable_char("ㅗ")
        (None, "ㅗ", None)
        >>> split_syllable_char("ㅇ")
        ("ㅇ", None, None)
    """
    check_hangul(c)
    if len(c) != 1:
        raise ValueError("Input string must have exactly one character.")

    init, med, final = None, None, None
    if is_hangul_syllable(c):
        offset = ord(c) - 0xac00
        x = (offset - offset % 28) // 28
        init, med, final = x // 21, x % 21, offset % 28
        if not final:
            final = None
        else:
            final -= 1
    else:
        pos = get_jamo_type(c)
        if pos & INITIAL == INITIAL:
            pos = INITIAL
        elif pos & MEDIAL == MEDIAL:
            pos = MEDIAL
        elif pos & FINAL == FINAL:
            pos = FINAL
        idx = CHAR_INDICES[pos][c]
        if pos == INITIAL:
            init = idx
        elif pos == MEDIAL:
            med = idx
        else:
            final = idx
    return tuple(CHAR_LISTS[pos][idx] if idx is not None else None
                 for pos, idx in
                 zip([INITIAL, MEDIAL, FINAL], [init, med, final]))


def split_syllables(s, ignore_err=True, pad=None):
    """
    Performs syllable-split on a string.

    Arguments:
        s (str): A string (possibly mixed with non-Hangul characters).
        ignore_err (bool): If set False, it ensures that all characters in
            the string are Hangul-splittable and throws a ValueError otherwise.
            (default: True)
        pad (str): Pad empty jamo positions (initial, medial, or final) with
            `pad` character. This is useful for cases where fixed-length
            strings are needed. (default: None)

    Returns:
        Hangul-split string

    Example:
        >>> split_syllables("안녕하세요")
        "ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ"
        >>> split_syllables("안녕하세요~~", ignore_err=False)
        ValueError: encountered an unsupported character: ~ (0x7e)
        >>> split_syllables("안녕하세요ㅛ", pad="x")
        'ㅇㅏㄴㄴㅕㅇㅎㅏxㅅㅔxㅇㅛxxㅛx'
    """

    def try_split(c):
        try:
            return split_syllable_char(c)
        except ValueError:
            if ignore_err:
                return (c,)
            raise ValueError(f"encountered an unsupported character: "
                             f"{c} (0x{ord(c):x})")

    s = map(try_split, s)
    if pad is not None:
        tuples = map(lambda x: tuple(pad if y is None else y for y in x), s)
    else:
        tuples = map(lambda x: filter(None, x), s)
    return "".join(itertools.chain(*tuples))


def join_jamos_char(init, med, final=None):
    """
    Combines jamos into a single syllable.

    Arguments:
        init (str): Initial jao.
        med (str): Medial jamo.
        final (str): Final jamo. If not supplied, the final syllable is made
            without the final. (default: None)

    Returns:
        A Korean syllable.
    """
    chars = (init, med, final)
    for c in filter(None, chars):
        check_hangul(c, jamo_only=True)

    idx = tuple(CHAR_INDICES[pos][c] if c is not None else c
                for pos, c in zip((INITIAL, MEDIAL, FINAL), chars))
    init_idx, med_idx, final_idx = idx
    # final index must be shifted once as
    # final index with 0 points to syllables without final
    final_idx = 0 if final_idx is None else final_idx + 1
    return chr(0xac00 + 28 * 21 * init_idx + 28 * med_idx + final_idx)


def join_jamos(s, ignore_err=True):
    """
    Combines a sequence of jamos to produce a sequence of syllables.

    Arguments:
        s (str): A string (possible mixed with non-jamo characters).
        ignore_err (bool): If set False, it will ensure that all characters
            will be consumed for the making of syllables. It will throw a
            ValueError when it fails to do so. (default: True)

    Returns:
        A string

    Example:
        >>> join_jamos("ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ")
        "안녕하세요"
        >>> join_jamos("ㅇㅏㄴㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ")
        "안ㄴ녕하세요"
        >>> join_jamos()
    """
    last_t = 0
    queue = []
    new_string = ""

    def flush(n=0):
        new_queue = []
        while len(queue) > n:
            new_queue.append(queue.pop())
        if len(new_queue) == 1:
            if not ignore_err:
                raise ValueError(f"invalid jamo character: {new_queue[0]}")
            result = new_queue[0]
        elif len(new_queue) >= 2:
            try:
                result = join_jamos_char(*new_queue)
            except (ValueError, KeyError):
                # Invalid jamo combination
                if not ignore_err:
                    raise ValueError(f"invalid jamo characters: {new_queue}")
                result = "".join(new_queue)
        else:
            result = None
        return result

    for c in s:
        if c not in CHARSET:
            if queue:
                new_c = flush() + c
            else:
                new_c = c
            last_t = 0
        else:
            t = get_jamo_type(c)
            new_c = None
            if t & FINAL == FINAL:
                if not (last_t == MEDIAL):
                    new_c = flush()
            elif t == INITIAL:
                new_c = flush()
            elif t == MEDIAL:
                if last_t & INITIAL == INITIAL:
                    new_c = flush(1)
                else:
                    new_c = flush()
            last_t = t
            queue.insert(0, c)
        if new_c:
            new_string += new_c
    if queue:
        new_string += flush()
    return new_string

In [ ]:
import re

chars_to_ignore_regex = '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    batch['text'] = split_syllables(batch['text'])
    return batch

In [ ]:
import re

chars_to_ignore_regex = '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
zeroth_data = zeroth_data.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
show_random_elements(zeroth_data["train"].remove_columns(["file", "audio"]), num_examples=10)

,text
0,ㅇㅣ ㅅㅏㄱㅓㄴㅇㅡㄴ ㅅㅏㅁㅅㅣㅂ ㅍㅏㄹ ㄴㅕㄴㅇㅣ ㅈㅣㄴㅏㄴ ㄷㅟㅇㅣㄴ ㅇㅣㅊㅓㄴ ㅅㅣㅂ ㅇㅣ ㄴㅕㄴ ㄱㅜ ㅇㅝㄹ ㅈㅐㅅㅣㅁㅇㅡㄹ ㅌㅗㅇㅎㅐ ㅁㅜㅈㅚㄹㅡㄹ ㅅㅓㄴㄱㅗㅂㅏㄷㅇㅏㅆㄷㅏ
1,ㅎㅏㅂㅊㅏㅁㅇㅢ ㅇㅖㅅㅏㅇㅇㅣ ㅁㅏㅈㄷㅏㅁㅕㄴ ㅇㅣㅂㅓㄴ ㅂㅏㄹㅅㅏㄴㅡㄴ ㅇㅗㄹㅎㅐ ㄷㅡㄹㅇㅓ ㅇㅕㄷㅓㄼ ㅂㅓㄴㅉㅐ ㅅㅣㅎㅓㅁㅂㅏㄹㅅㅏㄷㅚㄴ ㅁㅜㅅㅜㄷㅏㄴ ㅁㅣㅅㅏㅇㅣㄹㅇㅣ ㄷㅚㄴㄷㅏ
2,ㄱㅡㄹㅓㄴ ㅇㅜㄹㅕㄱㅏ ㅇㅣㅆㅇㅓㅆㄷㅏㅁㅕㄴ ㅈㅐㅍㅏㄴㅇㅡㄹ ㅅㅣㅂ ㅇㅣㄹ ㄴㅕㄴㄱㅏㄴ ㄲㅡㄹㅇㅓㅇㅘㅆㄴㅡㄴㄷㅔㅇㅛ
3,ㅇㅠㅈㅗㄱㅇㅡㄴ ㅇㅖㅅㄴㅏㄹㅇㅔㄴㅡㄴ ㅎㅣㅁㄷㅡㄹㅇㅓㅆㅈㅣㅁㅏㄴ ㅎㅜㅇㅣㅁㅇㅣ ㄷㅡㄹㅇㅓㅇㅘ ㅈㅗㅎㄷㅏㅁㅕ ㄱㅣㅂㅜㄴ ㅈㅗㅎㄱㅔ ㅂㅗㄱㄱㅟㅎㅐㅆㄷㅓㄴ ㅇㅏㅇㅣㄱㅏ ㅂㅜㄷㅐ ㅂㅗㄱㄱㅟ ㅎㅏㄴ ㅅㅣㄱㅏㄴㅇㅕ ㅁㅏㄴㅇㅔ ㅅㅣㅅㅣㄴㅇㅡㄹㅗ ㅂㅏㄹㄱㅕㄴㄷㅚㄴ ㄱㅔ ㄴㅏㅂㄷㅡㄱㄷㅚㅈㅣ ㅇㅏㄶㄴㅡㄴㄷㅏㄱㅗ ㅎㅓㅁㅏㅇㅎㅐ ㅎㅐㅆㄷㅏ
4,ㅌㅡㄱㅎㅣ ㅇㅣ ㅇㅟㅇㅝㄴㅈㅏㅇㅇㅡㄴ ㄱㅣㅁㄷㅐㅈㅜㅇ ㅈㅓㅇㅅㅣㄴㅇㅡㄴ ㄴㅜㄱㅜㄷㅗ ㅅㅏㅇㅠㅎㅘ ㅎㅏㄹ ㅅㅜ ㅇㅓㅄㄱㅗ ㄷㅗㄱㅈㅓㅁㅎㅐㅅㅓㄴ ㅇㅏㄴ ㄷㅚㄴㄷㅏ
5,ㄱㅡㄹㅐㅅㅓ ㅊㅓㄴ ㄱㅜㅂㅐㄱ ㄱㅜㅅㅣㅂ ㅅㅏ ㄴㅕㄴ ㅁㅏㄹ ㅈㅡㅇㅡㅁㅇㅔ ㄴㅏㄷㅗ ㅇㅜㄴㄷㅗㅇㅇㅡㄹ ㅅㅣㅈㅏㄱㅎㅐㅆㅅㅡㅂㄴㅣㄷㅏ
6,ㅇㅕㅎㅗㅇㅘㅇㅢ ㅈㅡㅇㅇㅣㄴㅇㅡㄴ ㄱㅓㄴㄱㅏㅇㅇㅡㄹ ㄷㅗㄹㅂㅗㄹㅕㄱㅗ ㅇㅐㅆㅡㅁㅇㅡㄹㅗ ㅅㅐㅇㅁㅕㅇㅇㅡㄹ ㅅㅗㅈㅜㅇㅎㅣ ㅇㅕㄱㅣㄱㅗ ㅇㅣㅆㅇㅡㅁㅇㅡㄹ ㄴㅏㅌㅏㄴㅐㄴㄷㅏ
7,ㅈㅣㄴㅏㄴ ㅅㅏㅁ ㄴㅕㄴㄱㅏㄴ ㅅㅗㅂㅣㅈㅏㅇㅝㄴㅇㅔ ㅈㅓㅂㅅㅜㄷㅚㄴ ㅇㅣㅁㅍㅡㄹㄹㅏㄴㅌㅡ ㄱㅘㄴㄹㅕㄴ ㅍㅣㅎㅐㄱㅜㅈㅔ ㅅㅏㄱㅓㄴㅇㅢ ㄱㅜㅅㅣㅂ ㅍㅓㅅㅔㄴㅌㅡ ㅇㅣㅅㅏㅇㅇㅡㄴ ㅂㅜㅈㅏㄱㅇㅛㅇ ㄸㅐㅁㅜㄴㅇㅣㅇㅓㅆㅅㅡㅂㄴㅣㄷㅏ
8,ㅋㅣ ㅂㅐㄱ ㅇㅕㄷㅡㄴ ㅅㅔㄴㅌㅣㅁㅣㅌㅓㄱㅏ ㄴㅓㅁㄴㅡㄴ ㅊㅔㄱㅕㄱㅇㅔ ㅈㅏㄱㄷㅣ ㅈㅏㄱㅇㅡㄴ ㅊㅏㄱㅏ ㅂㅜㄹㅍㅕㄴㅎㅏㄹ ㄱㅓㅅ ㄱㅏㅌㅇㅏㅆㅈㅣㅁㅏㄴ ㄱㅡㄴㅡㄴ ㅈㅓㄴㅎㅕ ㅂㅜㄹㅍㅕㄴㅎㅏㅈㅣ ㅇㅏㄶㄱㅗ ㅇㅜㄴㅈㅓㄴㅇㅣ ㅈㅐㅁㅣㅇㅣㅆㄷㅏㄱㅗ ㅎㅐㅆㅅㅡㅂㄴㅣㄷㅏ
9,ㅋㅡㄹㅣㅅㅡㅁㅏㅅㅡㅇㅔ ㅅㅓㄴㅁㅜㄹㅇㅡㄹ ㅎㅏㄴㅡㄴ ㅈㅓㅇㅎㅘㄱㅎㅣ ㅁㅏㄹㅎㅏㅈㅏㅁㅕㄴ ㅅㅓㄴㅁㅜㄹㅇㅡㄹ ㅈㅜㄱㅗㅂㅏㄷㄴㅡㄴ ㄱㅘㄴㅅㅡㅂㅇㅡㄹ ㅎㅏㄴㅡㄴㅣㅁㄲㅔㅅㅓㄴㅡㄴ ㅇㅓㄸㅓㅎㄱㅔ ㅂㅗㅅㅣㅂㄴㅣㄲㅏ


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [ ]:
vocabs = zeroth_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=zeroth_data.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'ㄱ': 1,
 'ㄲ': 2,
 'ㄳ': 3,
 'ㄴ': 4,
 'ㄵ': 5,
 'ㄶ': 6,
 'ㄷ': 7,
 'ㄸ': 8,
 'ㄹ': 9,
 'ㄺ': 10,
 'ㄻ': 11,
 'ㄼ': 12,
 'ㅀ': 13,
 'ㅁ': 14,
 'ㅂ': 15,
 'ㅃ': 16,
 'ㅄ': 17,
 'ㅅ': 18,
 'ㅆ': 19,
 'ㅇ': 20,
 'ㅈ': 21,
 'ㅉ': 22,
 'ㅊ': 23,
 'ㅋ': 24,
 'ㅌ': 25,
 'ㅍ': 26,
 'ㅎ': 27,
 'ㅏ': 28,
 'ㅐ': 29,
 'ㅑ': 30,
 'ㅒ': 31,
 'ㅓ': 32,
 'ㅔ': 33,
 'ㅕ': 34,
 'ㅖ': 35,
 'ㅗ': 36,
 'ㅘ': 37,
 'ㅙ': 38,
 'ㅚ': 39,
 'ㅛ': 40,
 'ㅜ': 41,
 'ㅝ': 42,
 'ㅞ': 43,
 'ㅟ': 44,
 'ㅠ': 45,
 'ㅡ': 46,
 'ㅢ': 47,
 'ㅣ': 48}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

51

In [ ]:
import json
with open("vocab.json", 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-large-xls-r-300m-korean-e"

In [ ]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:42: FutureWarning: Deprecated positional argument(s) used in 'create_repo': pass token='wav2vec2-large-xls-r-300m-korean-e' as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:684: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-e into local empty directory.
To https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-e
   0cdfc99..8e974f1  main -> main

   0cdfc99..8e974f1  main -> main



'https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-e/commit/8e974f165ad6b99a57ce1d5442f054ce27f51389'

### Wav2Vec2 Feature Extractor

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Preprocess Data

In [ ]:
zeroth_data["train"][0]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/146/146_003_0005.flac'

In [ ]:
zeroth_data["train"][0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/146/146_003_0005.flac',
 'array': array([ 1.2207031e-04,  1.8310547e-04,  9.1552734e-05, ...,
        -1.2207031e-04,  1.8310547e-04, -1.2207031e-04], dtype=float32),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(zeroth_data["train"]))

print(zeroth_data["train"][rand_int]["text"])
ipd.Audio(data=zeroth_data["train"][rand_int]["audio"]["array"], autoplay=True, rate=16000)

ㅁㅣㄱㅜㄱㅇㅢ ㅂㅜㄱㅎㅏㄴ ㅁㅣㅅㅏㅇㅣㄹ ㅈㅓㄴㅁㅜㄴㄱㅏㅇㅣㄴ ㅈㅗㄴ ㅅㅣㄹㄹㅣㅇㅇㅡㄴ ㅂㅜㄱㅎㅏㄴㅇㅣ ㅇㅣㅂㅓㄴㅇㅔ ㅅㅏㅇㅛㅇㅎㅏㄴ ㅇㅜㄴㅂㅏㄴㅊㅔ ㄱㅘㅇㅁㅕㅇㅅㅓㅇㅇㅣ ㅇㅡㄴㅎㅏ ㅅㅏㅁ ㅎㅗ ㄹㅗㅋㅔㅅㄱㅘ ㄸㅗㄱㄱㅏㅌㅇㅡㄴ ㅇㅔㄴㅈㅣㄴㅇㅡㄹ ㅅㅏㅇㅛㅇㅎㅏㄴㅡㄴ ㄷㅡㅇ ㅇㅡㄴㅎㅏ ㅅㅏㅁ ㅎㅗㅇㅢ ㄱㅓㅇㅢ ㅂㅗㄱㅅㅏㅂㅗㄴ ㅅㅜㅈㅜㄴㅇㅣㄹㅏㄱㅗ ㅅㅓㄹㅁㅕㅇㅎㅐㅆㄷㅏ 


In [ ]:
rand_int = random.randint(0, len(zeroth_data["train"]))

print("Target text:",zeroth_data["train"][rand_int]["text"])
print("Input array shape:", np.asarray(zeroth_data["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", zeroth_data["train"][rand_int]["audio"]["sampling_rate"])

Target text: ㅅㅓㅇㅅㅓ ㅍㅣㄹㅈㅏㄷㅡㄹㅇㅡㄴ ㄸㅗㅎㅏㄴ ㅅㅏㅅㅗㅎㅏㄴ ㅈㅓㅁㅇㅔㅅㅓㄲㅏㅈㅣㄷㅗ ㅈㅓㅇㅎㅘㄱㅎㅏㅇㅕㅆㅅㅡㅂㄴㅣㄷㅏ 
Input array shape: (78832,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
zeroth_data = zeroth_data.map(prepare_dataset, remove_columns=zeroth_data.column_names["train"], num_proc=4)

In [ ]:
max_input_length_in_sec = 5.0
zeroth_data["train"] = zeroth_data["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/23 [00:00<?, ?ba/s]

## 5.&nbsp;Trainer 준비

Set-up Trainer

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
cer_metric = load_metric("cer")

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
from transformers import Wav2Vec2ForCTC

'''
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)
'''

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_hid.weight', 'project_q.bias', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1712: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
from transformers import TrainingArguments

'''
training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  dataloader_num_workers = 1,
  evaluation_strategy="steps",
  num_train_epochs=5,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=1e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)
'''

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=zeroth_data["train"],
    eval_dataset=zeroth_data["test"],
    tokenizer=processor.feature_extractor,
)

/content/wav2vec2-large-xls-r-300m-korean-e is already a clone of https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-e. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


##6.&nbsp;모델 훈련

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1230
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3080


Step,Training Loss,Validation Loss,Cer
500,5.382700,3.539073,1.000000
1000,3.063300,2.985444,0.875908
1500,1.209500,0.838430,0.210317
2000,0.549900,0.673288,0.168928
2500,0.381500,0.677756,0.159067
3000,0.311100,0.663974,0.151839


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 457
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xls-r-300m-korean-e/checkpoint-500
Configuration saved in wav2vec2-large-xls-r-300m-korean-e/checkpoint-500/config.json
Model weights saved in wav2vec2-large-xls-r-300m-korean-e/checkpoint-500/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xls-r-300m-korean-e/checkpoint-500/preprocessor_config.json
Feature extractor saved in wav2vec2-large-xls-r-300m-korean-e/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.

TrainOutput(global_step=3080, training_loss=1.7767797383395108, metrics={'train_runtime': 4540.6019, 'train_samples_per_second': 5.418, 'train_steps_per_second': 0.678, 'total_flos': 3.465436737100451e+18, 'train_loss': 1.7767797383395108, 'epoch': 20.0})

## 7.&nbsp;모델 저장

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xls-r-300m-korean-e
Configuration saved in wav2vec2-large-xls-r-300m-korean-e/config.json
Model weights saved in wav2vec2-large-xls-r-300m-korean-e/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xls-r-300m-korean-e/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.18G [00:00<?, ?B/s]

Upload file runs/Sep22_08-56-40_ec0805f72390/events.out.tfevents.1663837073.ec0805f72390.76.0:  43%|####2     …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-e
   8e974f1..3ce2bbb  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-e
   8e974f1..3ce2bbb  main -> main

Dropping the following result as it does not have all the necessary fields:
{}
To https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-e
   3ce2bbb..02c0cdf  main -> main

   3ce2bbb..02c0cdf  main -> main



'https://huggingface.co/thisisHJLee/wav2vec2-large-xls-r-300m-korean-e/commit/3ce2bbb832f24b7c8c91986f1d87e7a65f81a381'

## 8.&nbsp;모델 평가

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

loading feature extractor configuration file wav2vec2-large-xls-r-300m-korean-e/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

Didn't find file wav2vec2-large-xls-r-300m-korean-e/added_tokens.json. We won't load it.
loading file wav2vec2-large-xls-r-300m-korean-e/vocab.json
loading file wav2vec2-large-xls-r-300m-korean-e/tokenizer_config.json
loading file None
loading file wav2vec2-large-xls-r-300m-korean-e/special_tokens_map.json
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file wav2vec2-large-xls-r-300m-korean-e/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-xls-r-300

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = zeroth_data["test"].map(map_to_result, remove_columns=zeroth_data["test"].column_names)

0ex [00:00, ?ex/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
print("Test CER: {:.3f}".format(cer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test CER: 0.215


In [ ]:
show_random_elements(results)

,pred_str,text
0,ㅇㅇㅓㄱㅔ ㄱㅘㄴㄱㅔ ㅅㅏㄴㅣㄴ ㄱㅏㄱㅇㅕㄱㅣ ㅇㅗㄹㅡㅁㅕㅕㄴ ㅈㅣㅅㅏㅇㅂㅏㅏ ㅅㅏㄴ ㅈㅅㅏㄴㄴㅐ ㅇㅕㅇㅡㄴㄱㅏㅎㅏㄴ ㅊㅏㅏ ㅇㅂㅇㅔ ㅁㅇㅓㄱㅇㅝㄴ ㅊㅜㄱㅏㅅㅡㄹ ㅇㅣㄱㅡㄹ ㅇㅗㄹㄹㅇㅣㄴㄴㅏ ㅁㅕㄴㅅㅏ ㅇ ㄱㅡ ㅍㅜㅈㅏㅁㅇㅡㄴ ㅇㅣㅇㅇㅛㅈㅇㅣㄱㅔ ㅋㄱㅗㅅㅡㄹㅡㅣㅇㅣㄴ ㄴㅓㅇㅝㄱㅏㅡㄹ ㅅㅡㅂㅏㅈㅈㄱㅔ ㅇㅓㅂㄷㅏㄱㅗ ㄷㅗㄹㅗ ㅇㅣㅆㄷㄷㅏ,ㅇㅓㅂㄱㅖ ㄱㅘㄴㄱㅖㅈㅏㄴㅡㄴ ㄱㅏㄱㅕㄱㅇㅣ ㅇㅗㄹㅡㅁㅕㄴ ㅈㅣㅅㅏㅇㅍㅏ ㅅㅏㅁ ㅅㅏㄴㅡㄴ ㅇㅕㄴㄱㅏㄴ ㅊㅓㄴ ㅇㅣㅂㅐㄱ ㅇㅓㄱㅇㅝㄴ ㅊㅜㄱㅏ ㅅㅜㅇㅣㄱㅇㅡㄹ ㅇㅗㄹㄹㅣㄴㄷㅏㅁㅕㄴㅅㅓ ㄱㅡ ㅂㅜㄷㅏㅁㅇㅡㄴ ㅇㅣㅇㅛㅇㅈㅏㅇㅔㄱㅔ ㄱㅗㅅㅡㄹㅏㄴㅎㅣ ㄴㅓㅁㅇㅓㄱㅏㄹ ㅅㅜㅂㅏㄲㅇㅔ ㅇㅓㅄㄷㅏㄱㅗ ㅌㅗㄹㅗㅎㅐㅆㄷㅏ
1,ㄱㅡㄴㅡㄴ ㅇㅗㄱㅋㅏ ㅇㅓㅅㅔ ㄷㅐㅎㅏㄴ ㅎㅘㄱㄸㅐㅎㅐㅅㅓㄱㅇㅔ ㅅㅓㄴㅎㅔ ㄱㅡㅇㅡㅁㅕㄴㅅㅗㄷㅗ ㅅㅏㅇㅓㅂ ㄷㅏㄱㅏ ㄱㅘㅂㅕㄹ ㅇㅟㅎㅕㄴ ㅌㅜㅈㅏㅅㅣㄹㅍㅐㄹㅗ ㅈㅐㅇㅇㅜㄱㅜㅈㅓㄱㅏ ㅇㅏㅏㄷㅚㄴ ㄱㅓㅅㅇㅡㄴ ㅅㅏㅅㅣ ㅇㅣㄹㅏㄱㅗ ㄷㅓ ㅂㅗㅌㅇㅕㅆㄷㅏ,ㄱㅡㄴㅡㄴ ㅇㅝㅋㅡㅇㅏㅇㅜㅅㅇㅔ ㄷㅐㅎㅏㄴ ㅎㅘㄱㄷㅐ ㅎㅐㅅㅓㄱㅇㅔ ㅅㅓㄴㅇㅡㄹ ㄱㅡㅇㅡㅁㅕㄴㅅㅓㄷㅗ ㅅㅏㅇㅓㅂ ㄷㅏㄱㅏㄱㅎㅘㄹㅡㄹ ㅇㅟㅎㅏㄴ ㅌㅜㅈㅏ ㅅㅣㄹㅍㅐㄹㅗ ㅈㅐㅁㅜㄱㅜㅈㅗㄱㅏ ㅇㅏㄱㅎㅘㄷㅚㄴ ㄱㅓㅅㅇㅡㄴ ㅅㅏㅅㅣㄹㅇㅣㄹㅏㄱㅗ ㄷㅓㅅㅂㅜㅌㅇㅕㅆㄷㅏ
2,ㅁㅐㄷㅏㅇ ㅂㅗㅁㅝㄴㅇㅡㄴ ㅅㅏㅁㅊㅔㅌㅔ ㅈㅓㅇㄱㅏㄴㅇ ㄴㅡㄱㅏ ㅇㅣㄹㅗ ㅌㅈㅐㅣㅇㅇㅡㅁㅎㄱㅕㄱㅔㅆ ㅈㅣㅁㅏㄴ ㄴㅇㅣㄹㄹㅕㄴㄷㅗㅇㅏㄴㄷㅔㄱㅗㅇㅈㅡㄹㅣㅈㅏㄱㅗ ㄷㅇㅡㄹ ㄱㅓㅅㅇㅓㄷㅏㅏ,ㅎㅐㄷㅏㅇ ㄱㅗㅇㅁㅜㅇㅝㄴㅇㅡㄴ ㅅㅏㅁㅅㅣㅂ ㄷㅐ ㅈㅜㅇㅂㅏㄴㅇㅔ ㄴㅡㅈㄲㅏㄲㅇㅣㄹㅗ ㅅㅣㅎㅓㅁㅇㅔ ㅎㅏㅂㄱㅕㄱㅎㅐㅆㅈㅣㅁㅏㄴ ㅇㅣㄹ ㄴㅕㄴㄷㅗ ㅇㅏㄴ ㄷㅙ ㄱㅗㅇㅈㅣㄱㅈㅏ ㅇㅗㅅㅇㅡㄹ ㅂㅓㅅㅇㅓㅆㄷㅏ
3,ㄱㅡㄴㅡㄴ ㅅㅡㅋㅗㅌㅡㄹㄹㅔㄴㅏㅏ ㅇㅏㄹㄹㅕㄴㄷㅡㄹㅇㅡㄹ ㅇㅜㅇㅔㅅㅓ ㅅㅡㅍㅔㅎㅐㄱㅇㅡㄹㅗ ㄷㅗㄹㅇㅓㅎㅏㄱㅣㄹㅗ ㅂㅕㄹㅈㅓㅇㅎㅐㅆㅅㅡㅂㄴㅣㄷㅏ,ㄱㅡㄴㅡㄴ ㅅㅡㅋㅗㅌㅡㄹㄹㅐㄴㄷㅡㅇㅘ ㅇㅏㅇㅣㄹㄹㅐㄴㄷㅡㄹㅡㄹ ㅇㅜㅎㅚㅎㅐㅅㅓ ㅅㅡㅍㅔㅇㅣㄴㅇㅡㄹㅗ ㄷㅗㄹㅇㅏㄱㅏㄱㅣㄹㅗ ㄱㅕㄹㅈㅓㅇㅎㅐㅆㅅㅡㅂㄴㅣㄷㅏ
4,ㅁㅏㄴㅇㅣㄹ ㄴㅐㅓㄱㅔ ㅅㅏㄹㅏㄱㅁㅘㅏ ㄱㅛㄹㅓㄴㅎㅐ ㄸㅓㄹㅎㅏㅁㅕㄴ ㄱㅡㄹㅣ ㅅㅡㄷㅗ ㅇㅣㄴ ㅈㅣㅂㅇㅔㅔ ㅎㅗㄴㅈㅏㄷㅏㄴㅡㄹ ㅂㅓㄴㅎㅐㅆㅇㅓㅇㅛ,ㅁㅏㄴㅇㅣㄹ ㄴㅐㄱㅏ ㄱㅡ ㅅㅏㄹㅏㅁㄱㅘ ㄱㅕㄹㅎㅗㄴㅎㅐㅆㄷㅓㄹㅏㅁㅕㄴ ㄱㅡㄹㅣㅅㅡㄷㅗㅇㅣㄴ ㅈㅣㅂㅎㅚㅇㅔ ㅎㅗㄴㅈㅏ ㄷㅏㄴㅣㄹ ㅃㅓㄴㅎㅐㅆㅇㅓㅇㅛ
5,ㅇㅕㅇㄱㅜㅈㅣㄴㅡㄴ ㄴㅏㄴㅍㅚㄴㅇㅔㄱㅕㅇㅗ ㅇㅏㄷㅐㅇㅘㄴ ㅅㅏㄱㅏㅌㅡㄹㅇㅓㅅㅣㄴ ㅎㅗㅇㅚㄷㅗ ㅊㅜㅇㅂㅜㄴㅏ ㅅㅗㅁㅕ ㅅㅣㄱㅏㄴㅇㅣㅎㅏ ㅍㅗㄷㅐㅇㄱㅕㅇㅡㄹㅏㅁㅕㄴ ㅏㄴㅐㅇㅇㅘㅏㅇㅢ ㄱㅕㄹㅗㄴㅅㅔㅏㅁㅏㄹㅎㅇㅔ ㄷㅐㅇㅔ ㄴㅏㄱㅏㄴㅈㅓㄱㅇㅣㄴ ㅈㅏㅈㅔㄹㅡㄹ ㅇㅠㅈㅣ ㅎㅏㄴㅡㄴ ㄱㅓㅅㅇㅡㄹㅣ ㄱㅏㄷㅇㅏㄹㅏㅆㄷㅏㄱㅗ ㅅㅓㄹㄹㅕㅎㅐㅆㄷㅏ,ㅇㅕㄴㄱㅜㅈㅣㄴㅇㅡㄴ ㄴㅏㅁㅍㅕㄴㅇㅢ ㄱㅕㅇㅇㅜ ㅇㅏㄴㅐㅇㅘ ㅅㅏㅇㅣㄱㅏ ㅌㅡㄹㅇㅓㅈㅣㄴ ㅎㅜㅇㅔㄷㅗ ㅊㅜㅇㅂㅜㄴㅎㅏㄴ ㅅㅜㅁㅕㄴㅅㅣㄱㅏㄴㅇㅣ ㅎㅘㄱㅂㅗㄷㅚㄴ ㄱㅕㅇㅇㅜㄹㅏㅁㅕㄴ ㅇㅏㄴㅐㅇㅘㅇㅢ ㄱㅕㄹㅎㅗㄴㅅㅐㅇㅎㅘㄹㅇㅔ ㄷㅐㅎㅐ ㄴㅏㄱㄱㅘㄴㅈㅓㄱㅇㅣㄴ ㅈㅏㅅㅔㄹㅡㄹ ㅇㅠㅈㅣㅎㅏㄴㅡㄴ ㄱㅓㅅㅇㅡㄹㅗ ㄴㅏㅌㅏㄴㅏㅆㄷㅏㄱㅗ ㅅㅓㄹㅁㅕㅇㅎㅐㅆㄷㅏ
6,ㅈㅣㄱㅣㅁ ㅇㅓㄷㅇㅣㄴㄱㅏㅇㅔㅅㅓ ㄴㅐㅇㅈㅓㅇㅎㅏㄱㅔ ㅇㅓㄹ ㅅㅣㅁㅇㅣㄹ ㅆㅣㄱㅗ ㅇㅣㅆㄷㅏㄴㅡㄴ ㅎㅏㄴㄱㅏㄱㅈㅏㄱㅏ ㄱㅡ ㄷㅏㅇㅡㅁㅈㅏㅍㅇㅡㅁ ㄱㅡ ㅈㅏㄱㅍㅜㅁㄷㅗㅎ ㄱㅣㅇㅑㄱㅗ ㅣㅎㅐㅆㅅㅡㅂㄴㅣㄷㅏ,ㅈㅣㄱㅡㅁ ㅇㅓㄷㅣㄴㄱㅏㅇㅔㅅㅓ ㄴㅐㅇㅈㅓㅇㅎㅏㄱㅔ ㅇㅕㄹㅅㅣㅁㅎㅣ ㅆㅡㄱㅗ ㅇㅣㅆㄷㅏㄴㅡㄴ ㅎㅏㄴㄱㅏㅇ ㅈㅏㄱㄱㅏㅇㅢ ㄱㅡ ㄷㅏㅇㅡㅁ ㅈㅏㄱㅍㅜㅁ ㄱㅡ ㅈㅏㄱㅍㅜㅁㄷㅗ ㄱㅣㄷㅐㅎㅏㄱㅗ ㅇㅣㅆㄱㅔㅆㅅㅡㅂㄴㅣㄷㅏ
7,ㄱㅣㅇㅜㄱㅏ ㄸㅓ ㅈㅗㅎㅇㅗㄴ ㄱㅜㄱㅗㅅㅇㅔㅅㅓ ㄷㅐㅎㅏㄱㄱㅛㄹㅡㄹ ㄷㅏㄴㅣㄱㅗ ㅇㅜㄴㄷㅓㄱㅏ ㄱㅘㄴㄹㅕㄴㅡㄴ ㅂㅜㄴㅑㅇㅔㅅㅓ ㅇㅣㄹㅇㅡㄹ ㅎㅔ ㅅㅐㅇㄱㅏㄱㅣㅇㅓㅆㅅㅡㅂㄴㅣㄷㅏ,ㄱㅣㅎㅜㄱㅏ ㄷㅓ ㅈㅗㅎㅇㅡㄴ ㄱㅡㄱㅗㅅㅇㅔㅅㅓ ㄷㅐㅎㅏㄱㄱㅛㄹㅡㄹ ㄷㅏㄴㅣㄱㅗ ㅇㅜㄴㄷㅗㅇㄱㅘ ㄱㅘㄴㄹㅕㄴㄷㅚㄴ ㅂㅜㄴㅇㅑㅇㅔㅅㅓ ㅇㅣㄹㅇㅡㄹ ㅎㅏㄹ ㅅㅐㅇㄱㅏㄱㅇㅣㅇㅓㅆㅅㅡㅂㄴㅣㄷㅏ
8,ㅈㅣㄴㅏㄴ ㅎㅐㅈㅏㅈㅏㅇㅈㅏㅇㅃㅗㅇㄹㅏㅁㅕㄴㅇㅡㄹㅗ ㅈㅜㅇㅇㅘ ㅇㅠㄹㅣ ㅈㅓㄴㅈㅐㅇㅇㅡㄹ ㅂㅓㄹㄹㅣㄴㄷㅔ ㅇㅣㅇㅓ ㅜㅜㄱㅏㄹ ㅂㅜㅐㅊㅣㄱㅇㅔ ㅇㅏㅁㅕㄴㅇㅡㄹㅗ ㄷㅏㅅㅣ ㅇㅏㄶㅂㅓㄴ ㅍㅜㄹㅇㅣㅣㅎㅓㅁ ㅈㅐㅍㅜㄴ ㅁㅏㅈㄱㅔㅆㅌㅣㄴ ㄱㅕㅇㅈㅐㅇㅇㅣ ㅇㅖㄹㄱㅗ ㄴㅙㅆㄷㅏ,ㅈㅣㄴㅏㄴㅎㅐ ㅉㅏㅈㅏㅇ ㅉㅏㅁㅃㅗㅇ ㄹㅏㅁㅕㄴㅇㅡㄹㅗ ㅈㅜㅇㅎㅘㅇㅛㄹㅣ ㅈㅓㄴㅈㅐㅇㅇㅡㄹ ㅂㅓㄹㅇㅣㄴㄷㅔ ㅇㅣㅇㅓ ㅇㅗㄹ ㄱㅏㅇㅡㄹ ㅂㅜㄷㅐㅉㅣㄱㅐ ㄹㅏㅁㅕㄴㅇㅡㄹㅗ ㄷㅏㅅㅣ ㅎㅏㄴ ㅂㅓㄴ ㅍㅡㄹㅣㅁㅣㅇㅓㅁ ㅈㅔㅍㅜㅁ ㅁㅏㅋㅔㅌㅣㅇ ㄱㅕㅇㅈㅐㅇㅇㅣ ㅇㅖㄱㅗㄷㅙㅆㄷㅏ
9,ㅁㅐㅈㅏㅇ ㄱㅗㄱㄱㅗㅅㅔㄴㅡㄴ ㄸㅐㅇㅋㅠㅅㅡㅣㄹ ㅇㅣㅂㄱㅏㄴㅡㄴ ㄱㅏㄴㄱㅘ ㅇㅕㅇㅇㅓㅂㅈㅗㅇㅇㅛ ㅇㅏㄴㄴㅐ ㅁㅜㄴㅇㅣ ㅅㅔㅇㅓㅈㅓㅆㄷㅏ,ㅁㅐㅈㅏㅇ ㄱㅗㅅㄱㅗㅅㅇㅔㄴㅡㄴ ㄸㅐㅇㅋㅠ ㅅㅔㅇㅣㄹ ㅇㅣㅂㄱㅏㄴㅍㅏㄴㄱㅘ ㅇㅕㅇㅇㅓㅂㅈㅗㅇㄹㅛ ㅇㅏㄴㄴㅐㅁㅜㄴㅇㅣ ㅅㅔㅇㅝㅈㅕㅆㄷㅏ


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(zeroth_data["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ㅁ [PAD] [PAD] ㅗ [PAD] [PAD] ㄴ [PAD] [PAD] [PAD] [PAD] ㅌ [PAD] [PAD] ㅗ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ㄱ ㄱ [PAD] ㅇ ㅇ ㅇ [PAD] ㅠ [PAD] [PAD] [PAD] [PAD] ㄴ [PAD] ㅇ ㅡ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ㄴ | | | [PAD] [PAD] ㅈ [PAD] [PAD] [PAD] [PAD] ㅏ [PAD] [PAD] [PAD] [PAD] ㄴ ㄴ [PAD] [PAD] ㅕ ㅕ [PAD] [PAD] [PAD] | | [PAD] ㄷ ㅡ ㅐ [PAD] [PAD] [PAD] [PAD] ㄹ ㅇ ㅇ ㅣ [PAD] [PAD] [PAD] [PAD] | | | [PAD] ㅅ [PAD] [PAD] ㅏ ㅏ [PAD] [PAD] [PAD] ㄹ ㄹ [PAD] [PAD] [PAD] ㅏ [PAD] [PAD] ㅇ [PAD] [PAD] ㅇ ㅇ ㅡ [PAD] [PAD] ㄴ [PAD] [PAD] [PAD] | | | [PAD] ㅈ ㅈ [PAD] ㅔ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ㄷ [PAD] ㅐ [PAD] [PAD] [PAD] ㄹ [PAD] [PAD] ㅗ ㅗ [PAD] [PAD] | | [PAD] [PAD] ㅁ [PAD] ㅗ [PAD] [PAD] ㄱ [PAD] [PAD] [PAD] [PAD] [PAD] ㅂ [PAD] [PAD] ㅏ [PAD] ㄷ ㄷ [PAD] [PAD] [PAD] [PAD] [PAD] ㄱ